In [ ]:
import os
from dotenv import load_dotenv 
load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")
print("Groq API key loaded successfully" if groq_api_key else "Warning: GROQ_API_KEY not found in environment variables")

In [ ]:
from langchain_groq import ChatGroq

model = ChatGroq(
    model_name="Gemma2-9b-it",
    groq_api_key=groq_api_key,
)
model

In [ ]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hello, I am an AI Engineer")])

In [ ]:
from langchain_core.messages import AIMessage

model.invoke([HumanMessage(content="Hello, I am an AI Engineer"),
 AIMessage(content="Hello,Yash Its nice to meet you, what do you do as an AI Engineer?"),
 HumanMessage(content="Whats my name and what do i do")])

In [ ]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(model, get_session_history)
config = {"configurable": {"session_id": "chat1"}}

In [ ]:
response = with_message_history.invoke([HumanMessage(content="Hello, I am Yash working as an AI Engineer")], config=config)
response.content

In [ ]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder

# Create a dynamic prompt template that accepts language as input
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant, answer questions to best of your ability in {language}"),
    MessagesPlaceholder(variable_name="messages")
])

# Create a chain that accepts both messages and language
chain = prompt | model

In [ ]:
# Test the chain directly with language parameter
chain.invoke({
    "messages": [HumanMessage(content="Hello, I am Yash working as an AI Engineer")],
    "language": "Telugu"
})

In [ ]:
# Create a function to handle language-specific conversations
def create_language_chain(language):
    """Create a chain with a specific language"""
    language_prompt = ChatPromptTemplate.from_messages([
        ("system", f"You are a helpful assistant, answer questions to best of your ability in {language}"),
        MessagesPlaceholder(variable_name="messages")
    ])
    return language_prompt | model

# Create chains for different languages
telugu_chain = create_language_chain("Telugu")
english_chain = create_language_chain("English")

# Create with_message_history for each language
telugu_with_history = RunnableWithMessageHistory(telugu_chain, get_session_history)
english_with_history = RunnableWithMessageHistory(english_chain, get_session_history)

In [ ]:
# Using Telugu chain with message history
response = telugu_with_history.invoke(
    {"messages": [HumanMessage(content="Hello, I am Yash working as an AI Engineer")]}, 
    config={"configurable": {"session_id": "telugu_session"}}
)
response.content

In [ ]:
# Example: Using different languages
# English response
response_en = english_with_history.invoke(
    {"messages": [HumanMessage(content="What is artificial intelligence?")]}, 
    config={"configurable": {"session_id": "english_session"}}
)
print("English Response:", response_en.content)

# Telugu response  
response_te = telugu_with_history.invoke(
    {"messages": [HumanMessage(content="What is artificial intelligence?")]}, 
    config={"configurable": {"session_id": "telugu_session_2"}}
)
print("Telugu Response:", response_te.content)

In [ ]:
# Function to create a language-specific chat with history
def chat_in_language(language, message, session_id=None):
    """
    Create a chat response in the specified language
    
    Args:
        language: The language for responses (e.g., "Telugu", "English", "Hindi")
        message: The user's message
        session_id: Optional session ID for conversation history
    """
    if session_id is None:
        session_id = f"{language.lower()}_session"
    
    # Create chain for this language
    lang_chain = create_language_chain(language)
    lang_with_history = RunnableWithMessageHistory(lang_chain, get_session_history)
    
    # Get response
    response = lang_with_history.invoke(
        {"messages": [HumanMessage(content=message)]}, 
        config={"configurable": {"session_id": session_id}}
    )
    
    return response.content

# Example usage:
print("Telugu Response:")
print(chat_in_language("Telugu", "Hello, how are you?", "test_telugu"))

print("\nEnglish Response:")
print(chat_in_language("English", "Hello, how are you?", "test_english"))

print("\nHindi Response:")
print(chat_in_language("Hindi", "Hello, how are you?", "test_hindi"))

In [ ]:
from langchain_core.messages import SystemMessage, trim_messages

trimmer = trim_messages(
    max_tokens=100,
    strategy= "last",
    token_counter = model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)

messages = [
    SystemMessage(content="You are a helpful assistant that provides friendly advice about food and desserts."),
    
    HumanMessage(content="Hi! I'm craving something sweet. What do you suggest?"),
    
    AIMessage(content="How about treating yourself to some ice cream? It's a classic and always hits the spot."),
    
    HumanMessage(content="That sounds perfect! Any flavor you recommend?"),
    
    AIMessage(content="Vanilla is always a safe bet, but if you're feeling adventurous, try salted caramel or pistachio."),
    
    HumanMessage(content="Where can I buy good ice cream nearby?"),
    
    AIMessage(content="You can check out your local grocery store, or look for artisanal ice cream shops in your area. Yelp or Google Maps can help!"),
    
    HumanMessage(content="Thanks! I'll head out now and get some."),
    
    AIMessage(content="Enjoy your ice cream! Let me know if you need anything else.")
]

trimmer.invoke(messages)

In [ ]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain =(
    RunnablePassthrough.assign(
        messages = itemgetter("messages")|trimmer
    ) | prompt | model
)

response = chain.invoke({"messages": messages+[HumanMessage(content="What is artificial intelligence?")], "language": "Telugu"})
response.content

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()
from langchain_groq import ChatGroq

groq_api_key = os.getenv("GROQ_API_KEY")
os.environ["HF_TOKEN"] = os.getenv("HF_TOKEN")
llama_model = ChatGroq(
    model_name="Llama3-8b-8192",
    groq_api_key=groq_api_key,
)
llama_model

In [ ]:
from langchain_core.documents import Document

# Create dummy documents for testing
documents = [
    Document(
        page_content="Artificial Intelligence (AI) is a branch of computer science that aims to create machines capable of intelligent behavior. It involves developing algorithms and systems that can perform tasks typically requiring human intelligence, such as learning, reasoning, and problem-solving.",
        metadata={"source": "ai_basics.txt", "topic": "AI Introduction", "difficulty": "beginner"}
    ),
    Document(
        page_content="Machine Learning is a subset of AI that focuses on algorithms that can learn and improve from experience without being explicitly programmed. It uses statistical techniques to enable computers to learn from data and make predictions or decisions.",
        metadata={"source": "ml_basics.txt", "topic": "Machine Learning", "difficulty": "intermediate"}
    ),
    Document(
        page_content="Deep Learning is a subset of machine learning that uses artificial neural networks with multiple layers to model and understand complex patterns in data. It has been particularly successful in image recognition, natural language processing, and speech recognition.",
        metadata={"source": "dl_basics.txt", "topic": "Deep Learning", "difficulty": "advanced"}
    ),
    Document(
        page_content="Natural Language Processing (NLP) is a field of AI that focuses on the interaction between computers and humans through natural language. It involves developing systems that can understand, interpret, and generate human language in a valuable way.",
        metadata={"source": "nlp_basics.txt", "topic": "NLP", "difficulty": "intermediate"}
    ),
    Document(
        page_content="Computer Vision is a field of AI that trains computers to interpret and understand the visual world. Using digital images from cameras and videos and deep learning models, machines can accurately identify and classify objects and react to what they see.",
        metadata={"source": "cv_basics.txt", "topic": "Computer Vision", "difficulty": "intermediate"}
    ),
    Document(
        page_content="Python is a high-level programming language that is widely used in AI and machine learning development. It has a simple syntax and a large ecosystem of libraries like TensorFlow, PyTorch, and scikit-learn that make it ideal for AI projects.",
        metadata={"source": "python_ai.txt", "topic": "Programming", "difficulty": "beginner"}
    ),
    Document(
        page_content="TensorFlow is an open-source machine learning framework developed by Google. It provides a comprehensive ecosystem of tools, libraries, and community resources that lets researchers and developers build and deploy ML applications.",
        metadata={"source": "tensorflow.txt", "topic": "Frameworks", "difficulty": "intermediate"}
    ),
    Document(
        page_content="PyTorch is an open-source machine learning library based on the Torch library. It's known for its dynamic computational graph and is widely used in research and production for deep learning applications.",
        metadata={"source": "pytorch.txt", "topic": "Frameworks", "difficulty": "intermediate"}
    ),
    Document(
        page_content="Data preprocessing is a crucial step in machine learning that involves cleaning and transforming raw data into a format that can be used by machine learning algorithms. This includes handling missing values, encoding categorical variables, and scaling features.",
        metadata={"source": "data_preprocessing.txt", "topic": "Data Science", "difficulty": "beginner"}
    ),
    Document(
        page_content="Neural networks are computing systems inspired by biological neural networks. They consist of interconnected nodes (neurons) that process information using a connectionist approach to computation, learning to perform tasks by considering examples.",
        metadata={"source": "neural_networks.txt", "topic": "Neural Networks", "difficulty": "intermediate"}
    )
]

print(f"Created {len(documents)} dummy documents")
for i, doc in enumerate(documents):
    print(f"Document {i+1}: {doc.metadata['topic']} - {doc.metadata['difficulty']}")

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

# Create embeddings
embeddings = HuggingFaceEmbeddings(
    model_name="all-MiniLM-L6-v2"
)
print("Embeddings model loaded successfully")

In [ ]:
from langchain_chroma import Chroma

vector_store = Chroma.from_documents(documents, embedding=embeddings)
print(f"Vector store created with {vector_store._collection.count()} documents")

In [ ]:
retriever = vector_store.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.5}
)

# Test retrieval
results = retriever.batch(["NLP", "Machine Learning"])
print(f"Retrieved {len(results[0])} documents for 'NLP' query")
print(f"Retrieved {len(results[1])} documents for 'Machine Learning' query")